# 注意力机制

In [3]:
import torch

# 创建一个简单的例子
def simple_attention_example():
    # 创建查询和键向量
    query = torch.tensor([[2.0, 3.0]])  # 1x2
    key = torch.tensor([[1.0, 2.0],
                       [0.5, 1.0]])      # 2x2
    
    # 计算注意力分数
    scores = torch.matmul(query, key.T)  # 1x2
    print("原始分数:", scores)
    
    # 缩放
    d_k = query.size(-1)  # = 2
    scaled_scores = scores / torch.sqrt(torch.tensor(d_k).float())
    print("缩放后的分数:", scaled_scores)
    
    # 应用softmax
    attention_weights = torch.nn.functional.softmax(scaled_scores, dim=-1)
    print("注意力权重:", attention_weights)
    
    return attention_weights

# 运行示例
attention_weights = simple_attention_example()

原始分数: tensor([[8., 4.]])
缩放后的分数: tensor([[5.6569, 2.8284]])
注意力权重: tensor([[0.9442, 0.0558]])


In [5]:
nopeak_mask = (1 - torch.triu(torch.ones(1, 4, 4), diagonal=1)).bool()

print(nopeak_mask)

tensor([[[ True, False, False, False],
         [ True,  True, False, False],
         [ True,  True,  True, False],
         [ True,  True,  True,  True]]])


In [1]:
import numpy as np

class SimpleAttention:
    def __init__(self, input_dim, hidden_dim):
        # 初始化权重矩阵
        self.Wq = np.random.randn(input_dim, hidden_dim) / np.sqrt(input_dim)  # Query权重
        self.Wk = np.random.randn(input_dim, hidden_dim) / np.sqrt(input_dim)  # Key权重
        self.Wv = np.random.randn(input_dim, hidden_dim) / np.sqrt(input_dim)  # Value权重
        self.Wo = np.random.randn(hidden_dim, input_dim) / np.sqrt(hidden_dim)  # 输出权重
        
    def softmax(self, x):
        # 实现softmax函数
        exp_x = np.exp(x - np.max(x, axis=-1, keepdims=True))  # 减去最大值防止数值溢出
        return exp_x / np.sum(exp_x, axis=-1, keepdims=True)
    
    def forward(self, X):
        """
        X: 输入序列，shape (batch_size, seq_len, input_dim)
        """
        # 1. 生成Query、Key、Value
        Q = np.dot(X, self.Wq)  # (batch_size, seq_len, hidden_dim)
        K = np.dot(X, self.Wk)  # (batch_size, seq_len, hidden_dim)
        V = np.dot(X, self.Wv)  # (batch_size, seq_len, hidden_dim)
        
        # 2. 计算注意力分数
        scores = np.dot(Q, K.transpose(0, 2, 1))  # (batch_size, seq_len, seq_len)
        
        # 3. 缩放注意力分数
        scores = scores / np.sqrt(K.shape[-1])
        
        # 4. 应用softmax得到注意力权重
        attention_weights = self.softmax(scores)  # (batch_size, seq_len, seq_len)
        
        # 5. 加权求和得到上下文向量
        context = np.dot(attention_weights, V)  # (batch_size, seq_len, hidden_dim)
        
        # 6. 线性变换得到输出
        output = np.dot(context, self.Wo)  # (batch_size, seq_len, input_dim)
        
        return output, attention_weights

# 测试代码
def test_attention():
    # 设置参数
    batch_size = 2
    seq_len = 4
    input_dim = 6
    hidden_dim = 8
    
    # 创建输入数据
    X = np.random.randn(batch_size, seq_len, input_dim)
    
    # 初始化注意力层
    attention = SimpleAttention(input_dim, hidden_dim)
    
    # 前向传播
    output, attention_weights = attention.forward(X)
    
    # 打印结果
    print("输入形状:", X.shape)
    print("输出形状:", output.shape)
    print("注意力权重形状:", attention_weights.shape)
    print("\n第一个样本的注意力权重矩阵:")
    print(attention_weights[0])
    
    return output, attention_weights

# 运行测试
if __name__ == "__main__":
    output, weights = test_attention()

输入形状: (2, 4, 6)
输出形状: (2, 4, 2, 2, 6)
注意力权重形状: (2, 4, 2, 4)

第一个样本的注意力权重矩阵:
[[[0.06442246 0.05851506 0.012118   0.86494448]
  [0.07135287 0.0096341  0.59548844 0.32352459]]

 [[0.04042779 0.322952   0.49723829 0.13938192]
  [0.14238434 0.72594244 0.0502511  0.08142212]]

 [[0.00665577 0.22435428 0.49451061 0.27447934]
  [0.17749634 0.36344675 0.08819053 0.37086638]]

 [[0.50402078 0.18061427 0.26243141 0.05293353]
  [0.22724647 0.55528546 0.12544975 0.09201832]]]


In [1]:
import torch
import torch.nn as nn
import math
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence

def create_masks(src, tgt):
    # 创建源序列掩码
    src_mask = (src != 0).unsqueeze(1).unsqueeze(2)
    
    if tgt is None:  # 在推理时tgt可能为None
        return src_mask, None
    
    # 创建目标序列掩码
    tgt_mask = (tgt != 0).unsqueeze(1).unsqueeze(2)
    seq_length = tgt.size(1)
    
    # 创建自回归掩码
    nopeak_mask = (1 - torch.triu(torch.ones((1, seq_length, seq_length)), diagonal=1)).bool()
    tgt_mask = tgt_mask & nopeak_mask.to(tgt.device)
    
    return src_mask, tgt_mask

class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super().__init__()
        assert d_model % num_heads == 0
        
        self.d_model = d_model
        self.num_heads = num_heads
        self.d_k = d_model // num_heads
        
        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)
        self.W_o = nn.Linear(d_model, d_model)
        
    def scaled_dot_product_attention(self, Q, K, V, mask=None):
        scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)
        
        if mask is not None:
            scores = scores.masked_fill(mask == 0, float('-inf'))
        
        attention_weights = torch.softmax(scores, dim=-1)
        output = torch.matmul(attention_weights, V)
        return output, attention_weights
    
    def split_heads(self, x):
        batch_size = x.size(0)
        x = x.view(batch_size, -1, self.num_heads, self.d_k)
        return x.transpose(1, 2)
    
    def forward(self, Q, K, V, mask=None):
        batch_size = Q.size(0)
        
        Q = self.W_q(Q)
        K = self.W_k(K)
        V = self.W_v(V)
        
        Q = self.split_heads(Q)
        K = self.split_heads(K)
        V = self.split_heads(V)
        
        output, attention_weights = self.scaled_dot_product_attention(Q, K, V, mask)
        
        output = output.transpose(1, 2).contiguous().view(batch_size, -1, self.d_model)
        
        output = self.W_o(output)
        return output, attention_weights

class PositionwiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff):
        super().__init__()
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        return self.fc2(self.relu(self.fc1(x)))

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_seq_length=5000):
        super().__init__()
        
        pe = torch.zeros(max_seq_length, d_model)
        position = torch.arange(0, max_seq_length, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)
        
    def forward(self, x):
        return x + self.pe[:, :x.size(1)]

class TransformerEncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout=0.1):
        super().__init__()
        
        self.mha = MultiHeadAttention(d_model, num_heads)
        self.ffn = PositionwiseFeedForward(d_model, d_ff)
        
        self.layernorm1 = nn.LayerNorm(d_model)
        self.layernorm2 = nn.LayerNorm(d_model)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, mask=None):
        attn_output, _ = self.mha(x, x, x, mask)
        x = self.layernorm1(x + self.dropout(attn_output))
        
        ffn_output = self.ffn(x)
        x = self.layernorm2(x + self.dropout(ffn_output))
        
        return x

class TransformerDecoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout=0.1):
        super().__init__()
        
        self.mha1 = MultiHeadAttention(d_model, num_heads)
        self.mha2 = MultiHeadAttention(d_model, num_heads)
        self.ffn = PositionwiseFeedForward(d_model, d_ff)
        
        self.layernorm1 = nn.LayerNorm(d_model)
        self.layernorm2 = nn.LayerNorm(d_model)
        self.layernorm3 = nn.LayerNorm(d_model)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, enc_output, src_mask=None, tgt_mask=None):
        attn1_output, _ = self.mha1(x, x, x, tgt_mask)
        x = self.layernorm1(x + self.dropout(attn1_output))
        
        attn2_output, _ = self.mha2(x, enc_output, enc_output, src_mask)
        x = self.layernorm2(x + self.dropout(attn2_output))
        
        ffn_output = self.ffn(x)
        x = self.layernorm3(x + self.dropout(ffn_output))
        
        return x

class Transformer(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length=5000, dropout=0.1):
        super().__init__()
        
        self.encoder_embedding = nn.Embedding(src_vocab_size, d_model)
        self.decoder_embedding = nn.Embedding(tgt_vocab_size, d_model)
        self.positional_encoding = PositionalEncoding(d_model, max_seq_length)
        
        self.encoder_layers = nn.ModuleList([
            TransformerEncoderLayer(d_model, num_heads, d_ff, dropout)
            for _ in range(num_layers)
        ])
        
        self.decoder_layers = nn.ModuleList([
            TransformerDecoderLayer(d_model, num_heads, d_ff, dropout)
            for _ in range(num_layers)
        ])
        
        self.fc = nn.Linear(d_model, tgt_vocab_size)
        self.dropout = nn.Dropout(dropout)
        
    def encode(self, src, src_mask):
        x = self.dropout(self.positional_encoding(self.encoder_embedding(src)))
        
        for encoder_layer in self.encoder_layers:
            x = encoder_layer(x, src_mask)
            
        return x
    
    def decode(self, tgt, enc_output, src_mask, tgt_mask):
        x = self.dropout(self.positional_encoding(self.decoder_embedding(tgt)))
        
        for decoder_layer in self.decoder_layers:
            x = decoder_layer(x, enc_output, src_mask, tgt_mask)
            
        output = self.fc(x)
        return output
    
    def forward(self, src, tgt, src_mask, tgt_mask):
        enc_output = self.encode(src, src_mask)
        dec_output = self.decode(tgt, enc_output, src_mask, tgt_mask)
        return dec_output

class SimpleDataset(Dataset):
    def __init__(self, num_samples=1000, max_len=10):
        self.data = []
        for _ in range(num_samples):
            length = np.random.randint(3, max_len)
            src = np.random.randint(1, 10, size=length)
            tgt = np.flip(src).copy()
            # 添加开始符号（1）和结束符号（2）
            tgt = np.concatenate([[1], tgt, [2]])
            src = torch.LongTensor(src)
            tgt = torch.LongTensor(tgt)
            self.data.append((src, tgt))
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx]

def collate_fn(batch):
    src_batch, tgt_batch = zip(*batch)
    src_batch = pad_sequence(src_batch, batch_first=True, padding_value=0)
    tgt_batch = pad_sequence(tgt_batch, batch_first=True, padding_value=0)
    return src_batch, tgt_batch

def train_model(model, train_loader, criterion, optimizer, device, num_epochs):
    model.train()
    
    for epoch in range(num_epochs):
        total_loss = 0
        for batch_idx, (src, tgt) in enumerate(train_loader):
            src, tgt = src.to(device), tgt.to(device)
            
            tgt_input = tgt[:, :-1]
            tgt_output = tgt[:, 1:]
            
            src_mask, tgt_mask = create_masks(src, tgt_input)
            
            optimizer.zero_grad()
            output = model(src, tgt_input, src_mask, tgt_mask)
            
            loss = criterion(output.contiguous().view(-1, output.size(-1)), 
                           tgt_output.contiguous().view(-1))
            
            loss.backward()
            # 梯度裁剪，防止梯度爆炸
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            
            total_loss += loss.item()
            
            if batch_idx % 100 == 0:
                print(f'Epoch: {epoch}, Batch: {batch_idx}, Loss: {loss.item():.4f}')
        
        avg_loss = total_loss / len(train_loader)
        print(f'Epoch: {epoch}, Average Loss: {avg_loss:.4f}')

def predict(model, src, device, max_len=50):
    model.eval()
    src = torch.LongTensor(src).unsqueeze(0).to(device)
    src_mask, _ = create_masks(src, None)
    
    enc_output = model.encode(src, src_mask)
    
    # 开始符号
    output = torch.LongTensor([[1]]).to(device)  # 1 是开始符号
    
    for _ in range(max_len-1):
        _, tgt_mask = create_masks(src, output)
        
        dec_output = model.decode(output, enc_output, src_mask, tgt_mask)
        prob = torch.softmax(dec_output[:, -1], dim=-1)
        pred = prob.argmax(dim=-1)
        
        output = torch.cat([output, pred.unsqueeze(0)], dim=1)
        
        if pred.item() == 2:  # 2 是结束符号
            break
    
    return output.squeeze().cpu().numpy()

def main():
    # 设置随机种子以保证可重复性
    torch.manual_seed(42)
    np.random.seed(42)
    
    # 参数设置
    # 源语言词表大小，包括特殊token（如<PAD>, <START>, <END>等）
    src_vocab_size = 12  
    # 目标语言词表大小，包括特殊token
    tgt_vocab_size = 12
    # Transformer模型的维度，决定了词嵌入和各层的特征维度
    d_model = 128
    # 多头注意力机制中的头数，必须能被d_model整除
    num_heads = 8
    # Transformer编码器和解码器的层数
    num_layers = 3
    # 前馈神经网络的隐藏层维度
    d_ff = 512
    # Dropout比率，用于防止过拟合
    dropout = 0.1
    # 每个批次的样本数
    batch_size = 32
    # 训练的轮数
    num_epochs = 10
    # 训练设备，优先使用GPU，否则使用CPU
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    print(f"Using device: {device}")

    # 创建数据集和数据加载器
    dataset = SimpleDataset()
    train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)

    # 创建模型
    model = Transformer(
        src_vocab_size=src_vocab_size,
        tgt_vocab_size=tgt_vocab_size,
        d_model=d_model,
        num_heads=num_heads,
        num_layers=num_layers,
        d_ff=d_ff,
        dropout=dropout
    ).to(device)

    total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"Total trainable parameters: {total_params}")
    
    # 定义损失函数和优化器
    criterion = nn.CrossEntropyLoss(ignore_index=0)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

    # 训练模型
    train_model(model, train_loader, criterion, optimizer, device, num_epochs)

    # 测试示例
    test_input = [3, 1, 4, 1, 5]
    print(f"Input sequence: {test_input}")
    output = predict(model, test_input, device)
    print(f"Predicted sequence: {output}")

if __name__ == "__main__":
    main()

Using device: cpu
Total trainable parameters: 1393164
Epoch: 0, Batch: 0, Loss: 2.6125
Epoch: 0, Average Loss: 2.0882
Epoch: 1, Batch: 0, Loss: 1.8289
Epoch: 1, Average Loss: 1.6931
Epoch: 2, Batch: 0, Loss: 1.5439
Epoch: 2, Average Loss: 1.5102
Epoch: 3, Batch: 0, Loss: 1.4064
Epoch: 3, Average Loss: 1.4020
Epoch: 4, Batch: 0, Loss: 1.3777
Epoch: 4, Average Loss: 1.3074
Epoch: 5, Batch: 0, Loss: 1.3067
Epoch: 5, Average Loss: 1.1816
Epoch: 6, Batch: 0, Loss: 1.1069
Epoch: 6, Average Loss: 1.0490
Epoch: 7, Batch: 0, Loss: 0.9880
Epoch: 7, Average Loss: 0.9109
Epoch: 8, Batch: 0, Loss: 0.8771
Epoch: 8, Average Loss: 0.8041
Epoch: 9, Batch: 0, Loss: 0.6553
Epoch: 9, Average Loss: 0.7156
Input sequence: [3, 1, 4, 1, 5]
Predicted sequence: [1 5 1 4 1 3 2]


In [21]:
print(torch.arange(0, 20, 2))

tensor([ 0,  2,  4,  6,  8, 10, 12, 14, 16, 18])
